In [1]:
import tensorflow as tf
import pandas as pd
import numpy as np

from datetime import datetime, timedelta, timezone

import sys
sys.path.append('../')
from utils import data
from models.lenet import LenetTF as Lenet
from utils.visualization import *

In [ ]:
bn = True
# const
taskName = 'fasionClassfiction'
modelName = 'lenetTF{}'.format('_bn' if bn else '')
JST = timezone(timedelta(hours=+9), 'JST')
now = datetime.now(JST)
nowStr = now.strftime("%Y%m%d_%H%M%S")
showImgCount = 10

imgHeight = 224
imgWidth = 224
imgChannel = 3
imgShape = [-1, imgHeight, imgWidth, imgChannel]
labelSize = 13
tfrecordPath = './img/fashionDataset/tfrecord/dataset224.tfrecord'
trainRatio = 0.7

# hyper parameter
bs = 32
lr = 0.00001
ep = 50

logDir = '../logs/{}/{}/{}/'.format(taskName, modelName, nowStr)
metadataDir = '{}metadata.tsv'.format(logDir)
checkPointDir = '{}images.ckpt'.format(logDir)

In [ ]:
# main
with tf.Graph().as_default():
    # load data
    with tf.variable_scope('tfrecord'):
        tfrecord = data.TFRecord(tfrecordPath, labelSize)
        dataset = tfrecord.toDataset()
        # split dataset
        tfrecord = data.TFRecord(tfrecordPath, labelSize)
        dataset = tfrecord.toDataset()
        trainSize, testSize, trainDataset, testDataset = tfrecord.splitDataset(bs)
        trainIteration = trainSize // bs
        testIteration = testSize // bs        
        # TODO: data augmentation

    # make iterator
    with tf.variable_scope('train_data'):
        trainIterator = trainDataset.make_initializable_iterator()
        trainNextBatch = trainIterator.get_next(name='train_next_batch')
        trainIteratorInitOp = trainIterator.initializer
    with tf.variable_scope('test_data'):
        testIterator = testDataset.make_initializable_iterator()
        testNextBatch = testIterator.get_next(name='test_next_batch')
        testIteratorInitOp = testIterator.initializer

    
    x = tf.placeholder("float", [None, imgHeight, imgWidth, imgChannel], name='x')
    y = tf.placeholder("float", [None, labelSize], name='label')
    isTrain = tf.placeholder(tf.bool, name='is_train')
    
    network = Lenet(x, inputReshapeTo=imgShape, labelSize=labelSize, batchNorm=bn, visualization=True)
    _ = network.inference(isTrain)
    cost = network.loss(y)
    optimizer = tf.train.AdamOptimizer(lr)
    accuracy = network.accuracy(y)
    trainOp = network.training(cost, optimizer)
    
    saver = tf.train.Saver()
    config = tf.ConfigProto(gpu_options=tf.GPUOptions(allow_growth=True))
    sess = tf.Session(config=config)
    summaryWriter = tf.summary.FileWriter(logDir, graph=sess.graph)
    sess.run(tf.global_variables_initializer())
    # Training cycle
    for e in range(ep):
        trainAvgAcc = trainAvgLoss = testAvgAcc = testAvgLoss = 0.
        # train
        sess.run(trainIteratorInitOp)
        for i in range(trainIteration):
            _, batchX, batchY = sess.run(trainNextBatch)
            _, l, a = sess.run([trainOp, cost, accuracy], feed_dict={x: batchX, y: batchY, isTrain: True})
            trainAvgAcc += a / trainIteration
            trainAvgLoss += l / trainIteration

        # validation
        sess.run(testIteratorInitOp)
        for i in range(testIteration):
            _, testBatchX, testBatchY = sess.run(testNextBatch)
            l, a = sess.run([cost, accuracy], feed_dict={x: testBatchX, y: testBatchY, isTrain: False})
            testAvgAcc += a / testIteration
            testAvgLoss += l / testIteration

        # tensorBoard
        summaryWriter.add_summary(drawSclar('train', {'acc': trainAvgAcc, 'loss': trainAvgLoss}), global_step=e)
        summaryWriter.add_summary(drawSclar('validation', {'acc': testAvgAcc, 'loss': testAvgLoss}), global_step=e)
        saver.save(sess, checkPointDir, global_step=e)
        print('Epoch: {} | Train Accuracy: {:.4f} | Train Loss: {:.4f} | Test Accuracy: {:.4f} | Test Loss: {:.4f}'.format(e, trainAvgAcc, trainAvgLoss, testAvgAcc, testAvgLoss))
    summaryWriter.close()
    sess.close()